# Trees and Forests



## Training a Decision Tree Classifier

### Load Data From CSV File

In [56]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import math
from sklearn import preprocessing
%matplotlib inline

In [96]:
df = pd.read_csv('teleCust1000t.csv')
df.head()

,region,tenure,age,marital,address,income,ed,employ,retire,gender,reside,custcat
0,2,13,44,1,9,64.0,4,5,0.0,0,2,1
1,3,11,33,1,7,136.0,5,5,0.0,0,6,4
2,3,68,52,1,24,116.0,1,29,0.0,1,2,3
3,2,33,33,0,12,33.0,2,0,0.0,1,1,1
4,2,23,30,1,9,30.0,1,2,0.0,0,4,3


### Data Visualization and Analysis


In [97]:
df['custcat'].value_counts()

3    281
1    266
4    236
2    217
Name: custcat, dtype: int64

### Feature set

Lets define feature sets, X:

In [98]:
df.columns

Index(['region', 'tenure', 'age', 'marital', 'address', 'income', 'ed',
       'employ', 'retire', 'gender', 'reside', 'custcat'],
      dtype='object')

To use scikit-learn library, we have to convert the Pandas data frame to a Numpy array:

In [99]:
X = df[['region', 'tenure','age', 'marital', 'address', 'income', 'ed', 'employ','retire', 'gender', 'reside']] .values  #.astype(float)
X[0:5]


array([[  2.,  13.,  44.,   1.,   9.,  64.,   4.,   5.,   0.,   0.,   2.],
       [  3.,  11.,  33.,   1.,   7., 136.,   5.,   5.,   0.,   0.,   6.],
       [  3.,  68.,  52.,   1.,  24., 116.,   1.,  29.,   0.,   1.,   2.],
       [  2.,  33.,  33.,   0.,  12.,  33.,   2.,   0.,   0.,   1.,   1.],
       [  2.,  23.,  30.,   1.,   9.,  30.,   1.,   2.,   0.,   0.,   4.]])

In [100]:
y = df['custcat'].values
y[0:5]

array([1, 4, 3, 1, 3], dtype=int64)

### Train Test Split
Out of Sample Accuracy is the percentage of correct predictions that the model makes on data that that the model has NOT been trained on. Doing a train and test on the same dataset will most likely have low out-of-sample accuracy, due to the likelihood of being over-fit.

It is important that our models have a high, out-of-sample accuracy, because the purpose of any model, of course, is to make correct predictions on unknown data. So how can we improve out-of-sample accuracy? One way is to use an evaluation approach called Train/Test Split. Train/Test Split involves splitting the dataset into training and testing sets respectively, which are mutually exclusive. After which, you train with the training set and test with the testing set.

This will provide a more accurate evaluation on out-of-sample accuracy because the testing dataset is not part of the dataset that have been used to train the data. It is more realistic for real world problems.

In [101]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (800, 11) (800,)
Test set: (200, 11) (200,)


In [102]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix


decisiontree = DecisionTreeClassifier(random_state=0)
model = decisiontree.fit(X_train, y_train)

target_predicted=model.predict(X_test)
print("Accuracy", model.score(X_test, y_test))

matrix = confusion_matrix(y_test, target_predicted)
print("Class Confusion Matrix\n", matrix)



Accuracy 0.395
Class Confusion Matrix
 [[30  6 10  5]
 [ 7 19  8 10]
 [15 14 17  8]
 [18  9 11 13]]


### Discussion
Decision tree learners attempt to find a decision rule that produces the greatest decrease in impurity at a node. While there are a number of measurements of impurity, by default `DecisionTreeClassifier` uses Gini impurity:
$$
G(t) = 1 - \sum_{i=1}^c{p_i^2}
$$
where G(t) is the Gini impurity at node t and $p_i$ is the proportion of observations of class c at node t.

This process of finding the decision rules that create splits to increase impurity is repeated recursively untill all leaf nodes are pure (i.e. contain only one class) or some abritary cut-off is reached

We can change the `criterion` parameter to use a different impurity measurement

In [103]:
# create decision tree classifier using entropy
decisiontree_entropy = DecisionTreeClassifier(criterion='entropy', random_state=0)

model_entropy = decisiontree_entropy.fit(X_train, y_train)

target_predicted=model_entropy.predict(X_test)
print("Accuracy", model_entropy.score(X_test, y_test))

matrix = confusion_matrix(y_test, target_predicted)
print("Class Confusion Matrix\n", matrix)



Accuracy 0.335
Class Confusion Matrix
 [[23  9 10  9]
 [13 12  7 12]
 [14 16 17  7]
 [12 12 12 15]]


In [104]:
def calc_entropy(column):
    counts = np.bincount(column)

    probabilities = counts / len(column)
    
    entropy = 0
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

def calc_information_gain(data, split_name, target_name):

    original_entropy = calc_entropy(data[target_name])
    
    values = data[split_name].unique()
    
    
    left_split = data[data[split_name] == values[0]]
    right_split = data[data[split_name] == values[1]]
    
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    return original_entropy - to_subtract

In [ ]:
columns = ['region', 'potato_salad?', 'sushi?']

def highest_info_gain(columns):
  information_gains = {}
  
  for col in columns:
    information_gain = calc_information_gain(midwest, col, 'midwest?)                                       
    information_gains[col] = information_gain
                                        
  return max(information_gains, key=information_gains.get)
           

In [83]:
## Write your own code for decision tree by using Information Gain 
## Don't use ready code 


def calculate_column_IG(dataframe, column_name, class_label):

    IG = 0 

    if dataframe[column_name].dtypes != 'O':
        print("1");
        raise Exception("Kategorik olmayan kolon goruldu.")
    else:
        print("2"); 
        kolonun_nunique_sayisi = dataframe[column_name].nunique()
        
        for j in range(kolonun_nunique_sayisi): 
            
            if dataframe.isna().values.any(): 
                dataframe = dataframe.dropna() 
            else:
                hesaplanmak_istenen_sutun = pd.DataFrame(dataframe.loc[dataframe[column_name] == dataframe.loc[:,column_name].unique()[j]][[column_name, class_label]])

                vc_of_his = pd.DataFrame(hesaplanmak_istenen_sutun.value_counts())
                
                Entropy_of_his = entropy_calculate(hesaplanmak_istenen_sutun, hesaplanmak_istenen_sutun[class_label].name)
                
                level = dataframe.loc[:,column_name].unique()[j]
                
                SUM = vc_of_his.xs(level)[0].sum()
                row_of_df = dataframe.shape[0]
                
                prob = SUM / row_of_df
                
                IG += (Entropy_of_his[-1] * prob)
                
                if j == kolonun_nunique_sayisi - 1:
                    Information_Gain_List.append(IG)
                    IG = 0 
                    
    return print(Information_Gain_List[-1])

In [92]:
for i in df.columns:
    i
    




##  Visualizing a Decision Tree Model

In [40]:
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Image
from sklearn import tree


## Write your own code to visualize tree with 4 levels
....
....
....



SyntaxError: invalid syntax (Temp/ipykernel_29248/2741142556.py, line 8)